# HyperNodes Telemetry & Observability Examples

This notebook demonstrates the telemetry and observability features of HyperNodes:
- **ProgressCallback**: Live progress bars
- **TelemetryCallback**: Distributed tracing with Logfire
- **Waterfall Charts**: Post-hoc analysis

## Setup

First, make sure you have the telemetry dependencies installed:
```bash
pip install 'hypernodes[telemetry]'
```

In [1]:
import time
import logfire
from hypernodes import node, Pipeline
from hypernodes.telemetry import ProgressCallback, TelemetryCallback

# Configure logfire without console output to avoid interfering with tqdm progress bars
logfire.configure(send_to_logfire=False, console=False)

## Example 1: Basic Pipeline with Progress

Simple pipeline showing live progress bars.

In [2]:
@node(output_name="doubled")
def double(x: int) -> int:
    """Double the input value."""
    time.sleep(0.5)  # Simulate work
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    """Add one to the value."""
    time.sleep(0.3)  # Simulate work
    return doubled + 1


# Create pipeline with progress callback
pipeline = Pipeline(nodes=[double, add_one], callbacks=[ProgressCallback()])

result = pipeline.run(inputs={"x": 5})
print(f"\nResult: {result}")

Pipeline: pipeline_4455260208   0%|          | 0/2 [00:00<?, ?it/s]

double   0%|          | 0/1 [00:00<?, ?it/s]

add_one   0%|          | 0/1 [00:00<?, ?it/s]


Result: {'doubled': 10, 'result': 11}


## Example 2: Progress + Telemetry Together

Combine progress bars with telemetry tracing.

In [3]:
@node(output_name="preprocessed")
def preprocess(text: str) -> str:
    """Clean and preprocess text."""
    time.sleep(0.2)
    return text.strip().lower()


@node(output_name="tokens")
def tokenize(preprocessed: str) -> list:
    """Split text into tokens."""
    time.sleep(0.3)
    return preprocessed.split()


@node(output_name="count")
def count_tokens(tokens: list) -> int:
    """Count the tokens."""
    time.sleep(0.1)
    return len(tokens)


# Create callbacks
progress = ProgressCallback()
telemetry = TelemetryCallback()

# Compose them
pipeline = Pipeline(
    nodes=[preprocess, tokenize, count_tokens], callbacks=[progress, telemetry]
)

result = pipeline.run(inputs={"text": "  Hello World from HyperNodes!  "})
print(f"\nResult: {result}")

Pipeline: pipeline_4456028528   0%|          | 0/3 [00:00<?, ?it/s]

preprocess   0%|          | 0/1 [00:00<?, ?it/s]

tokenize   0%|          | 0/1 [00:00<?, ?it/s]

count_tokens   0%|          | 0/1 [00:00<?, ?it/s]


Result: {'preprocessed': 'hello world from hypernodes!', 'tokens': ['hello', 'world', 'from', 'hypernodes!'], 'count': 4}


## Example 3: Waterfall Chart Visualization

Generate an interactive waterfall chart showing execution timeline.

In [4]:
# Define inner pipeline nodes
@node(output_name="cleaned")
def clean_data(data: str) -> str:  # Changed from 'raw' to 'data'
    """Clean the raw data."""
    time.sleep(0.2)
    return data.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    """Normalize the data."""
    time.sleep(0.3)
    return cleaned.lower()


# Create inner pipeline
inner_pipeline = Pipeline(nodes=[clean_data, normalize], id="preprocessing")


# Define outer pipeline nodes
@node(output_name="data")
def load_data(source: str) -> str:
    """Load data from source."""
    time.sleep(0.1)
    return f"  DATA FROM {source}  "


@node(output_name="final_result")
def aggregate(normalized: str) -> dict:
    """Aggregate the results."""
    time.sleep(0.2)
    return {"processed": normalized, "length": len(normalized)}


# Create outer pipeline
telemetry_nested = TelemetryCallback()

outer_pipeline = Pipeline(
    nodes=[load_data, inner_pipeline, aggregate],
    callbacks=[ProgressCallback(), telemetry_nested],
    id="main_pipeline",
)

result = outer_pipeline.run(inputs={"source": "database"})
print(f"\nResult: {result}")

Pipeline: main_pipeline   0%|          | 0/3 [00:00<?, ?it/s]

load_data   0%|          | 0/1 [00:00<?, ?it/s]

aggregate   0%|          | 0/1 [00:00<?, ?it/s]


Result: {'data': '  DATA FROM database  ', 'cleaned': 'DATA FROM database', 'normalized': 'data from database', 'final_result': {'processed': 'data from database', 'length': 18}}


## Example 4: Nested Pipelines

Demonstrate hierarchical progress and tracing with nested pipelines.

In [5]:
# Define inner pipeline nodes
@node(output_name="cleaned")
def clean_data(raw: str) -> str:
    """Clean the raw data."""
    time.sleep(0.2)
    return raw.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    """Normalize the data."""
    time.sleep(0.3)
    return cleaned.lower()


# Create inner pipeline
inner_pipeline = Pipeline(nodes=[clean_data, normalize], id="preprocessing")

In [6]:
inner_pipeline.visualize()

In [7]:
@node(output_name="final_result")
def aggregate(normalized: str) -> dict:
    """Aggregate the results."""
    time.sleep(0.2)
    return {"processed": normalized, "length": len(normalized)}


# Create outer pipeline
telemetry_nested = TelemetryCallback()

outer_pipeline = Pipeline(
    nodes=[inner_pipeline, aggregate],
    callbacks=[ProgressCallback(), telemetry_nested],
    id="main_pipeline",
)

In [8]:
outer_pipeline.visualize()

In [9]:
result = outer_pipeline.run(inputs={"raw": "database", "source": "database"})
print(f"\nResult: {result}")

Pipeline: main_pipeline   0%|          | 0/2 [00:00<?, ?it/s]

aggregate   0%|          | 0/1 [00:00<?, ?it/s]


Result: {'cleaned': 'database', 'normalized': 'database', 'final_result': {'processed': 'database', 'length': 8}}


### Nested Pipeline Waterfall Chart

Notice how the chart shows the hierarchy:

In [10]:
chart = telemetry_nested.get_waterfall_chart()
chart

## Example 5: Map Operations

Process multiple items with progress tracking and telemetry.

In [11]:
@node(output_name="squared")
def square(x: int) -> int:
    """Square a number."""
    time.sleep(0.1)  # Simulate processing
    return x * x


@node(output_name="result")
def is_even(squared: int) -> bool:
    """Check if squared result is even."""
    time.sleep(0.05)
    return squared % 2 == 0


# Create pipeline for mapping
telemetry_map = TelemetryCallback(trace_map_items=False)  # Don't trace each item

map_pipeline = Pipeline(
    nodes=[square, is_even], callbacks=[ProgressCallback(), telemetry_map]
)

results = map_pipeline.map(inputs={"x": [1, 2, 3]}, map_over="x")

Pipeline: pipeline_4456294464 (map)   0%|          | 0/3 [00:00<?, ?it/s]

square   0%|          | 0/3 [00:00<?, ?it/s]

is_even   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
print(f"\nProcessed {len(results)} items")


Processed 2 items


## Example 6: Testing with Disabled Progress

For automated testing, you can disable progress bars:

In [13]:
@node(output_name="result")
def simple_task(x: int) -> int:
    """Simple computation."""
    return x + 1


# Disabled progress (useful for tests)
progress_disabled = ProgressCallback(enable=False)

test_pipeline = Pipeline(nodes=[simple_task], callbacks=[progress_disabled])

result = test_pipeline.run(inputs={"x": 10})
print(f"Result (no progress shown): {result}")

Result (no progress shown): {'result': 11}


## Example 7: Complex Pipeline with Multiple Stages

A more realistic example with multiple processing stages.

In [14]:
@node(output_name="loaded")
def load_text(filename: str) -> str:
    """Simulate loading text from file."""
    time.sleep(0.2)
    return f"Sample text from {filename}"


@node(output_name="cleaned")
def clean_text(loaded: str) -> str:
    """Clean the text."""
    time.sleep(0.3)
    return loaded.lower().strip()


@node(output_name="tokens")
def extract_tokens(cleaned: str) -> list:
    """Extract tokens."""
    time.sleep(0.4)
    return cleaned.split()


@node(output_name="features")
def extract_features(tokens: list) -> dict:
    """Extract features from tokens."""
    time.sleep(0.5)
    return {
        "token_count": len(tokens),
        "unique_tokens": len(set(tokens)),
        "avg_length": sum(len(t) for t in tokens) / len(tokens) if tokens else 0,
    }


@node(output_name="analysis")
def analyze(features: dict) -> dict:
    """Analyze the features."""
    time.sleep(0.3)
    return {
        **features,
        "complexity_score": features["unique_tokens"] / features["token_count"]
        if features["token_count"] > 0
        else 0,
    }


# Create complex pipeline
telemetry_complex = TelemetryCallback()

complex_pipeline = Pipeline(
    nodes=[load_text, clean_text, extract_tokens, extract_features, analyze],
    callbacks=[ProgressCallback(), telemetry_complex],
    id="text_analysis",
)

result = complex_pipeline.run(inputs={"filename": "document.txt"})
print(f"\nAnalysis Result:")
for key, value in result.items():
    if key == "analysis":
        print(f"  {key}:")
        for k, v in value.items():
            print(f"    {k}: {v:.3f}" if isinstance(v, float) else f"    {k}: {v}")
    else:
        print(f"  {key}: {value}")

Pipeline: text_analysis   0%|          | 0/5 [00:00<?, ?it/s]

load_text   0%|          | 0/1 [00:00<?, ?it/s]

clean_text   0%|          | 0/1 [00:00<?, ?it/s]

extract_tokens   0%|          | 0/1 [00:00<?, ?it/s]

extract_features   0%|          | 0/1 [00:00<?, ?it/s]

analyze   0%|          | 0/1 [00:00<?, ?it/s]


Analysis Result:
  loaded: Sample text from document.txt
  cleaned: sample text from document.txt
  tokens: ['sample', 'text', 'from', 'document.txt']
  features: {'token_count': 4, 'unique_tokens': 4, 'avg_length': 6.5}
  analysis:
    token_count: 4
    unique_tokens: 4
    avg_length: 6.500
    complexity_score: 1.000


### Complex Pipeline Waterfall

Visualize the execution flow:

In [15]:
chart = telemetry_complex.get_waterfall_chart()
chart

## Chart Legend

**Waterfall Chart Colors:**
- 🟦 **Blue**: Regular nodes
- 🟧 **Orange**: Pipelines
- 🟩 **Green**: Cached operations or map operations

**Features:**
- Hover over bars to see details (duration, depth, type)
- Bars are arranged hierarchically (parent pipelines above children)
- Timeline shows parallel execution where applicable

## Summary

This notebook demonstrated:

1. ✅ **ProgressCallback** - Live progress bars with hierarchical display
2. ✅ **TelemetryCallback** - Distributed tracing with Logfire
3. ✅ **Waterfall Charts** - Interactive post-hoc analysis
4. ✅ **Nested Pipelines** - Automatic hierarchy handling
5. ✅ **Map Operations** - Batch processing with progress
6. ✅ **Testing Mode** - Disabling progress for automation

## Next Steps

- 📖 Read the full guide: `docs/guides/TELEMETRY_GUIDE.md`
- 🔗 Set up Logfire cloud: https://logfire.pydantic.dev
- 🧪 Run tests: `pytest tests/test_telemetry_*`
- 📊 Experiment with your own pipelines!